## Complete Market Case
1 risky asset, 2 random outcomes

$$ V_D^{(0)}={1\over1+\gamma}\left[\pi(\omega_1)V_D^{(1)}+\pi(\omega_2)V_D^{(2)}\right] $$

이 때 replicating portfolio : $\theta=(\theta_0, \theta_1)$ 에 대해서

$$ V_D^{(0)}=\theta_0+\theta_1S^{(0)}\\ V_D^{(i)}=\theta_0(1+\gamma)+\theta_1S^{(i)} $$

$$ \therefore \theta_0={1\over1+\gamma}\frac{V_D^{(1)}S^{(2)}-V_D^{(2)}S^{(1)}}{S^{(2)}-S^{(1)}}\quad\quad\quad \theta_1=\frac{V_D^{(2)}-V_D^{(1)}}{S^{(2)}-S^{(1)}} $$


## Incomplete Market Case
1 risky asset, 3 or more random asset

### Super/Subhedging

- Superhedging
    \begin{aligned}
    P_{\text{sup}}&=\min_\theta{\langle\theta, S^{(0)}\rangle}=\sup_\pi\left[{1\over1+\gamma}\pi(\omega_i)V_D^{(i)}\right]
    \end{aligned}
- Subhedging
    \begin{aligned}
    P_{\text{sub}}&=\max_\theta{\langle\theta, S^{(0)}\rangle}=\inf_\pi\left[{1\over1+\gamma}\pi(\omega_i)V_D^{(i)}\right]
    \end{aligned}



# Derivative Pricing for Incomplete Market

## Maximizing Utility Price
- **constraint**
    $t=0$에서 derivative $D$에 대한 비용지불 $x$, fundamental asset의 portfolio $\theta$에 대해서 다음이 성립
    \begin{aligned}
    $$ x + \theta_j S_j^{(0)} $$
    \end{aligned}

- Expected Utility
    \begin{aligned}
    L(V_D, x, \theta) = E[U] &= \sum_{i=0}^n\mu(\omega_i)\cdot U\left(V_D^{(i)}+\sum_{j=0}^m\theta_jS_j^{(i)}\right) \\
    &=\sum_{i=1}^n\mu(\omega_i)\cdot U\left(V_D^{(i)}-(1+\gamma)x+\sum_{j=1}^m\theta_j\left[S_j^{(i)}-(1+\gamma)S_j^{(0)}\right]\right)
    \end{aligned}


    $$ (\because \theta_0S_0^{(0)}=-\left(x-\theta_jS_j^{(0)}\right)) $$

- Breakeven value (손익분기점)
    손익분기점이 되는 derivative의 spot price $x^* $ 는 다음을 만족
    $$ \max_\theta{L(V_D, x^*;\theta)}=\max_\theta{L(0,0;\theta)} $$

## Simulation
- 1 risky asset
- price at $t=1$ follows normal distribution $ S \sim \mathcal{N}(\mu, \sigma^2) $
    => more than 2 random outcomes
    => incomplete market
- Constant Absolute Risk Aversion Utility function
    $$ U(y)={1-e^{-ay}\over a} $$

따라서 investment in the risky asset $\alpha$, investment in the riskless asset $\beta$, risky asset price $S$, payoff of derivative $f(S)$ 에 대하여,</br>
$$
x+\beta + \alpha S = 0 \quad\quad(\because S_0=1)\\
\max_\alpha E_S \left[{1-e^{-aL\left(f(S),x^*;\alpha\right)}\over a\right] =
\max_\alpha E_s \left[{1-e^{-aL\left(0,0;\alpha\right)}\over a}\right]
$$

where,
$$L\left(f(S),x^*;\alpha\right)=f(S)-(1+\gamma)x^*+\alpha(S-(1+\gamma)S_0) \\
L\left(0,0;\alpha\right)=\alpha(S-(1+\gamma)S_0)\\
\beta(\leftarrow \theta_0)  = -\left(x+\alpha S_0\right)$$

### Complete Market Price Method

$\alpha$는 investment기 때문에 holding $\theta$와 부호가 반대로, $ \theta_1 = -\alpha = {V^{(2)}-V^{(1)}\over S^{(2)}-S^{(1)}} $
$\beta$는 $V^{(i)} = \theta_0(1+\gamma) + \theta_1 S^{(i)}$ 로 부터, $\beta=-{V^{(i)}+\alpha S^{(i)}\over1+\gamma}$

`class MaxExpUtility`
- `risky_spot` : $S_0$
- `riskless_rate` : $\gamma$
- `risky_mean` : $\mu$
- `risky_stdev` : $\sigma$
- `payoff_func` : $f$

In [ ]:
import numpy as np

from typing import Callable, Mapping
from dataclasses import dataclass
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import minimize_scalar, root_scalar

@dataclass(frozen=True)
class MaxExpUtility:
    risky_spot: float
    riskless_rate: float
    risky_mean: float
    risky_stdev: float
    payoff_func: Callable[[float], float]

    def complete_market_price(self) -> Mapping[str, float]:
        """
        price changed to \mu+\sigma, \mu-\sigma with same possibility
        """
        x1 = self.risky_mean - self.risky_stdev
        x2 = self.risky_mean + self.risky_stdev
        v1 = self.payoff_func(x1)
        v2 = self.payoff_func(x2)
        alpha = -(v2 - v1)/(x2- x1)     # -theta_1
        beta = -(v2+alpha*x2)/(1+self.riskless_rate)    # -theta_0
        price = -(beta + alpha*self.risky_spot)

        return {'price': price, 'alpha': alpha, 'beta': beta}

    def max_expected_utility_price_with_zero(self, c: float, risk_aversion: float) -> Mapping[str, float]:
        """
        calculate max value for given position by maximizing the expected utility
        :param c: initial derivative price
        :param risk_aversion:  risk aversion param for CARA utility function
        :return:  max return value for given position
        """
        riskless_coeff = 1 + self.riskless_rate
        mu, sigma, spot = self.risky_mean, self.risky_stdev, self.risky_spot
        alpha = (mu - riskless_coeff*spot) / (risk_aversion * sigma**2)
        beta = - (c + alpha * spot)
        max_val = (1-np.exp(-risk_aversion*(
            -riskless_coeff*c+alpha*(mu-riskless_coeff*spot)+(risk_aversion*alpha*sigma)**2/2
        ))) / risk_aversion

        return {'max_val': max_val, 'alpha': alpha, 'beta': beta}

    def max_expected_utility_price(self, c: float, risk_aversion: float, payoff_fun: Callable[[float], float]=None):
        """
        calculate max value for given position with arbitrary payoff function
        :param c: initial derivative price
        :param risk_aversion:  risk aversion param for CARA utility function
        :param payoff_fun: payoff fnction
        :return:
        """
        if payoff_fun is None:
            return self.max_expected_utility_price(c, risk_aversion)

        mu, sigma, spot = self.risky_mean, self.risky_stdev, self.risky_spot
        riskless_coeff = 1 + self.riskless_rate

        lb = mu - sigma*6   # lower bound, 6-sigma
        ub = mu + sigma*6   # upper bound, 6-sigma

        def expectation(alpha):

            def integrand(s):
                # total payoff
                total_payoff = payoff_fun(s) - riskless_coeff*c + alpha*(s-riskless_coeff*spot)
                utility = (1-np.exp(-risk_aversion * total_payoff))/risk_aversion
                return norm.pdf(s, mu, sigma**2) * utility

            return